# Introdução

_Script_ que cria uma tabela que apresenta informações das \*Coordenadorias Regionais de Proteção e Defesa Civil - **REPDEC\*** da Defesa Civil que atuam nos 645 município deo Estado de São Paulo.


In [ ]:
#!pip3 install PyPDF2 --upgrade
#!pip3 install pdf2jpg --upgrade

In [ ]:
import os
import random
import shutil
import time
import urllib.parse
from zipfile import ZipFile

import geopandas as gpd
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from my_functions import rename_nome
from open_geodata import geo
from osgeo import gdal, ogr, osr
from pdf2jpg import pdf2jpg
from PyPDF2 import PdfFileReader, PdfFileWriter
from tqdm.notebook import tqdm, trange

from sp_defesacivil_divadmin.my_paths import (
    input_path_shp,
    output_path_gpkg,
    output_path_maps,
    output_path_tabs,
)

<br>

## Dados Espaciais: DataGeo


In [ ]:
# Input
url = 'http://datageo.ambiente.sp.gov.br/geoserver/datageo/RegionaisdaDefesaCivil/wfs?version=1.0.0&request=GetFeature&outputFormat=SHAPE-ZIP&typeName=RegionaisdaDefesaCivil'
filename = 'sp_defesacivil.zip'

r = requests.get(url, stream=True)
with open(input_path_shp / filename, 'wb') as out:
    for chunk in r.iter_content(chunk_size=128):
        out.write(chunk)

In [ ]:
# Lê arquivo
gdf = gpd.read_file(input_path_shp / filename)

# Transforma Coordenadas
gdf = gdf.to_crs(epsg=4326)

# Renomeia colunas
gdf = gdf.rename(
    columns={
        'Codigo': 'repdec_cod',
        'Nome': 'repdec_regiao',
    }
)

# Renomeia Campos Coluna
gdf.loc[:, 'repdec_cod'] = (
    gdf['repdec_cod'].astype(str).apply(lambda x: rename_nome(x))
)
gdf.loc[:, 'repdec_regiao'] = (
    gdf['repdec_regiao'].astype(str).apply(lambda x: rename_nome(x))
)

# Cria Cópia
gdf_repdec = gdf.copy()

# Results
gdf.info()
gdf.head()

In [ ]:
# Results
gdf.explore()

<br>

## Convert to Points


In [ ]:
# Load Nome dos Municípios
df_mun = geo.load_dataset('tab.sp.tab_municipio_nome')

# Read Geodataframe
gdf = geo.load_dataset('geo.sp.sp_050k_wgs84')
gdf = gdf.drop(['municipio_nome'], axis=1)
gdf = gdf.merge(df_mun, how='left', on='id_municipio')

# Cria Cópia
gdf_mun = gdf.copy()

# Results
gdf.info()
gdf.head()

In [ ]:
# Cria Cópia
gdf_centroid = gdf.copy()

# Pega centroid
gdf_centroid['geometry'] = gdf_centroid['geometry'].representative_point()

# Resuslts
gdf_centroid.explore()

<br>

## Spatial Join


In [ ]:
# Intersect
gdf_inter = gpd.sjoin(
    gdf_centroid, gdf_repdec, how='inner', predicate='intersects'
)

# Drop Columns
# gdf_inter = gdf_inter.drop(['index_right'], axis=1)
df_inter = gdf_inter[['id_municipio', 'repdec_cod', 'repdec_regiao']]

# Merge
gdf_inter = gdf_mun.merge(right=df_inter, how='left', on='id_municipio')

# Results
gdf_inter.explore(column='repdec_regiao')

<br>

## Join Defesa Civil

Por meio do [_site_ da Defesa Civil](http://www.defesacivil.sp.gov.br/) (mais específicamente nesse [_link_](http://www.defesacivil.sp.gov.br/coordenadores-regionais-de-defesa-civil/)) foi possível obter diversas informações sobre os Coordenadores que atuam nas 'Coordenadorias Regionais de Proteção e Defesa Civil - REPDEC'.

Foi utilizado o _script_ **get_infos.ipynb** para isso. Abaixo o arquivo gerado é apenas lido, para ser conctenado com a lista de municípios.


In [ ]:
# Lê o arquivo csv com o nome dos municípios
df_defesa = pd.read_csv(
    output_path_tabs / 'tab_defesacivil.csv',
)

# ddd
df_defesa.info()
df_defesa.head()

In [ ]:
# Merge
gdf_inter = gdf_inter.merge(
    df_defesa, how='left', left_on='repdec_cod', right_on='repdec_cod'
)

# Resultados
gdf_inter.info()
gdf_inter.head()

<br>

## Salva


In [ ]:
# Salva
gdf_inter.to_file(
    output_path_gpkg / 'sp_defesacivil.gpkg',
    driver='GPKG',
)